### 1. What are tools in langchain ?
        - Individual components that LangChain uses as links in the chain.

### 2. What are chains in langchain ?
        - How you link different tools ?

### 3. What is the benifit ?
        - you can combine multiple LLMs to create string LLMchain

### 4. Different types of Chains ?
        - Generic Chains 
                - LLMchain         -> the most common type of chain we build with PromptTemplates
                - TransformerChain -> Take something from your LLM (input or response) run a python 
                                      function on top of that [may be a regex or a custom function]
                - SequenceChain    -> Combine Multiple Chains
                
        - Utility Chains 
                - PAL Chain        -> Convert reasoning to a python code (may be internal) by LLM
                - SQL DB Chain     -> Natural Language Convert to SQL query and proceed with more work like
                                      retrieving data from DB and so on.
                (And there are tons of other utility chains)

        - Asynchronous Chains

In [10]:
import yaml, os, openai
from langchain import OpenAI
from langchain.chains import APIChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_experimental.pal_chain import PALChain
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import SimpleSequentialChain, LLMChain

In [5]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

## Basic LLMChain - Fact Extraction

In [6]:
llm = ChatOpenAI(
                openai_api_key=os.environ["OPENAI_API_KEY"],
                engine = os.environ["ENGINE"],
                model='gpt-3.5-turbo',
                temperature=0.9, 
                max_tokens = 256
                )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [7]:
article = '''Coinbase, the second-largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday, giving shareholders and market players alike an updated look into its financials. In response to the report, the company's shares are down modestly in early after-hours trading.In the fourth quarter of 2022, Coinbase generated $605 million in total revenue, down sharply from $2.49 billion in the year-ago quarter. Coinbase's top line was not enough to cover its expenses: The company lost $557 million in the three-month period on a GAAP basis (net income) worth -$2.46 per share, and an adjusted EBITDA deficit of $124 million.Wall Street expected Coinbase to report $581.2 million in revenue and earnings per share of -$2.44 with adjusted EBITDA of -$201.8 million driven by 8.4 million monthly transaction users (MTUs), according to data provided by Yahoo Finance.Before its Q4 earnings were released, Coinbase's stock had risen 86% year-to-date. Even with that rally, the value of Coinbase when measured on a per-share basis is still down significantly from its 52-week high of $206.79.That Coinbase beat revenue expectations is notable in that it came with declines in trading volume; Coinbase historically generated the bulk of its revenues from trading fees, making Q4 2022 notable. Consumer trading volumes fell from $26 billion in the third quarter of last year to $20 billion in Q4, while institutional volumes across the same timeframe fell from $133 billion to $125 billion.The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022, which resulted in Coinbase's total trading volumes and transaction revenues to fall 50% and 66% year-over-year, respectively, the company reported.As you would expect with declines in trading volume, trading revenue at Coinbase fell in Q4 compared to the third quarter of last year, dipping from $365.9 million to $322.1 million. (TechCrunch is comparing Coinbase's Q4 2022 results to Q3 2022 instead of Q4 2021, as the latter comparison would be less useful given how much the crypto market has changed in the last year; we're all aware that overall crypto activity has fallen from the final months of 2021.)There were bits of good news in the Coinbase report. While Coinbase's trading revenues were less than exuberant, the company's other revenues posted gains. What Coinbase calls its "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 of the same year, a gain of just over 34% in a single quarter.And even as the crypto industry faced a number of catastrophic events, including the Terra/LUNA and FTX collapses to name a few, there was still growth in other areas. The monthly active developers in crypto have more than doubled since 2020 to over 20,000, while major brands like Starbucks, Nike and Adidas have dived into the space alongside social media platforms like Instagram and Reddit.With big players getting into crypto, industry players are hoping this move results in greater adoption both for product use cases and trading volumes. Although there was a lot of movement from traditional retail markets and Web 2.0 businesses, trading volume for both consumer and institutional users fell quarter-over-quarter for Coinbase.Looking forward, it'll be interesting to see if these pieces pick back up and trading interest reemerges in 2023, or if platforms like Coinbase will have to keep looking elsewhere for revenue (like its subscription service) if users continue to shy away from the market.
'''

In [8]:
prompt = "Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep them short sentences. :\n\n {text_input}"
fact_extraction_prompt = PromptTemplate(
                                        input_variables=["text_input"],
                                        template=prompt
                                        )

In [9]:
fact_extraction_chain = LLMChain(
                                llm=llm, 
                                prompt=fact_extraction_prompt
                                )
facts = fact_extraction_chain.run(article)
print(facts)

1. Coinbase released its Q4 2022 earnings, showing a decline in revenue compared to the previous year.
2. Coinbase generated $605 million in total revenue in Q4 2022, down from $2.49 billion in the same quarter the previous year.
3. The company incurred a net loss of $557 million in the three-month period.
4. Wall Street expected Coinbase to report $581.2 million in revenue and a net loss per share of -$2.44.
5. Coinbase's stock had risen 86% year-to-date before the earnings report.
6. Coinbase beat revenue expectations despite declines in trading volume.
7. Consumer trading volumes fell from $26 billion to $20 billion, while institutional volumes fell from $133 billion to $125 billion.
8. The overall crypto market capitalization fell 64% in 2022.
9. Trading revenue at Coinbase fell from $365.9 million to $322.1 million in Q4 2022 compared to Q3 2022.
10. Coinbase's "subscription and services revenue" increased from $210.5 million to $282.8 million in Q4 2022.
11. The monthly active de

## Use the LLM again to rewrite as a summary from the facts

In [11]:
summary_update_prompt = PromptTemplate(
                                        input_variables=["facts"],
                                        template="You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:\n\n {facts}"
                                    )
summary_update_chain = LLMChain(llm=llm, prompt=summary_update_prompt)

summary_update = summary_update_chain.run(facts)

print(summary_update)

Dear investors,

We would like to provide you with an overview of Coinbase's Q4 2022 earnings report. Despite a decline in revenue compared to the previous year, with total revenue reaching $605 million, Coinbase faced a net loss of $557 million during this period. It is important to note that Wall Street had anticipated revenue to be $581.2 million. Prior to the release of the Q4 earnings, Coinbase's stock had risen by an impressive 86% year-to-date. However, trading volumes and transaction revenues experienced a significant decrease, falling by 50% and 66% year-over-year, respectively. Additionally, trading revenue in Q4 was lower when compared to the previous quarter. On a positive note, Coinbase's "subscription and services revenue" demonstrated a notable increase of over 34% in Q4 2022. In the broader crypto market, there has been significant growth, with monthly active developers more than doubling since 2020. Furthermore, renowned brands such as Starbucks, Nike, and Adidas have 

## Use the LLM again to rewrite in thriples format

In [13]:
triples_prompt = PromptTemplate(
                                input_variables=["facts"],
                                template="Take the following list of facts and turn them into triples for a knowledge graph:\n\n {facts}"
                                )

triples_chain = LLMChain(
                        llm=llm, 
                        prompt=triples_prompt
                        )
triples = triples_chain.run(facts)
print(triples)

1. Coinbase releasedEarnings inQ4 2022 : declined revenue compared to previous year
2. Coinbase generatedRevenue of $605 million inQ4 2022
3. Coinbase hadNet loss of $557 million inQ4 2022
4. Wall Street expected Coinbase to reportRevenue of $581.2 million
5. Coinbase stockRisen 86% year-to-date before Q4 earnings released
6. Coinbase trading volumesFell 50% year-over-year
7. Coinbase transaction revenuesFell 66% year-over-year
8. Coinbase trading revenueFell in Q4 compared to previous quarter
9. Coinbase "subscription and services revenue"Increased by over 34% inQ4 2022
10. Monthly active developers in cryptoMore than doubled since 2020
11. Major brands like Starbucks, Nike, and AdidasEntered the crypto space
12. Trading volume for both consumer and institutional usersFell quarter-over-quarter for Coinbase
13. Future trading interest and revenue generation in 2023Uncertain for Coinbase


### But the problem is you ran all 3 chains seperately, what if you want to run all 3 chains in one go ? Yeah SequenceChain

In [15]:
full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, summary_update_chain], verbose=True)
response = full_chain.run(article)
print(response)



> Entering new SimpleSequentialChain chain...
1. Coinbase released its Q4 2022 earnings, revealing its financials.
2. Coinbase generated $605 million in total revenue in Q4 2022, down from $2.49 billion in the previous year's quarter.
3. The company experienced a net loss of $557 million and an adjusted EBITDA deficit of $124 million in Q4 2022.
4. Wall Street expected Coinbase to report $581.2 million in revenue and negative earnings per share.
5. Coinbase's stock had risen 86% year-to-date before the earnings release.
6. Coinbase beat revenue expectations despite a decline in trading volume.
7. Consumer trading volumes fell from $26 billion to $20 billion, while institutional volumes fell from $133 billion to $125 billion in Q4 2022.
8. The overall crypto market capitalization fell by 64%, resulting in decreased trading volumes and transaction revenues for Coinbase.
9. Coinbase's trading revenue fell from $365.9 million to $322.1 million in Q4 2022.
10. Coinbase's "subscription and

# PAL Math Chain

In [ ]:
llm = OpenAI(
               model_name='code-davinci-002', 
               temperature=0, 
               max_tokens=512
               )

pal_chain = PALChain.from_math_prompt(llm, verbose=True)

question = "Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"

answer = pal_chain.run(question)
print(answer)